In [1]:
# https://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/PubTator/index.cgi?user=User622505252

In [2]:
from multiprocessing import Pool

In [3]:
num_cores = 4

In [4]:
chem = pd.read_csv('/Users/grace/workspace/bio_dataset/chemical2pubtator/chemical2pubtator.gz', sep='\t')

In [5]:
chem.head()

,PMID,MeshID,Mentions,Resource
0,1,MESH:D000432,methanol,tmChem|MESH
1,1,MESH:D005561,Formate,MESH
2,10,MESH:D004074,Digitoxin,MESH
3,1000,MESH:D009249,NADP,MESH
4,1000001,MESH:D002118,calcium,tmChem|MESH


### 전체 목록 28411962

In [6]:
chem.shape

(28411962, 4)

In [7]:
chem.dtypes

PMID         int64
MeshID      object
Mentions    object
Resource    object
dtype: object

In [8]:
chem.loc[0]

PMID                   1
MeshID      MESH:D000432
Mentions        methanol
Resource     tmChem|MESH
Name: 0, dtype: object

In [9]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_cores)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [45]:
import re
check = re.compile(r'tmchem| mesh')

# def check_removal_words(name):
#     if check.search(name) is None:
#         return name
#     else:
#         print(check.search(name))
#         return name

def remove_duplicates(item):
    m_arr = item.split('|')
    m_arr = list(set(filter(lambda x: x.strip(), m_arr)))
#     m_arr = list(map(lambda x: check_removal_words(x), m_arr))
    return ' , '.join(m_arr)

In [41]:
def convert_to_str(data):
    data['Resource'] = data['Resource'].apply(lambda x: str(x))
    data['Mentions'] = data['Mentions'].apply(lambda x: remove_duplicates(str(x)))
    return data

In [42]:
%time chem = parallelize_dataframe(chem, convert_to_str)

CPU times: user 15.9 ms, sys: 27.8 ms, total: 43.7 ms
Wall time: 130 ms


### MeSH 만 12213942

In [ ]:
chem = chem[chem.Resource.str.lower().str.find('mesh')>=0]

In [ ]:
chem.shape

## Mention에 MeSH, tcChem포함된 경우

In [44]:
chem[chem.Mentions.str.find('tcchem')>=0]

,PMID,MeshID,Mentions,Resource


### Unique 한 것만 396668

In [ ]:
chem.drop_duplicates(subset=['Mentions'], inplace=True)

In [ ]:
chem.shape

## 길이가 너무 짧은 약물 제외

In [ ]:
chem = chem[chem.Mentions.str.len()>4]

In [ ]:
chem.shape

In [ ]:
chem.to_csv('./mesh_only.tsv', sep='\t')